In [5]:
question = "Dấu hiệu của bệnh cảm cúm "
# context = "Bảo quản nơi khô ráo, thoáng mát.- Đặt thuốc ở nhiệt độ dưới 30°C và tránh ánh sáng.- Giữ thuốc xa tầm tay của trẻ em. Bài viết trên đã cung cấp những thông tin cơ bản nhất về thuốc Carsantin (Carsantin 6.25mg), Hãy gọi ngay cho bác sĩ hoặc đến ngay cơ sở y tế gần nhất để được hỗ trợ kịp thời khi xảy ra các triệu chứng không mong muốn hoặc các dấu hiệu có thể ảnh hưởng đến sức khỏe nhé!"
from Rag.redis_client import *
context = search_redis(question)
context

''

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import PeftModel
import torch

quantized_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Model/vnt5-qa-final")

# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    "PhucDanh/vit5-fine-tuning-for-question-answering",  # hoặc model gốc
    # device_map="auto",
    # quantization_config=quantized_config
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./Model/vnt5-qa-final")
model.eval()

input_text = f"Answer the question: {question} Context: {context}"

inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=384)
inputs = {k: v for k, v in inputs.items() if k != "token_type_ids"}

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=128)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Dấu hiệu của bệnh cảm cúm là dấu hiệu của bệnh cảm cúm.


NameError: name 'model' is not defined